In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd

import seaborn as sns

import json
import pathlib as pl

import numpy as np
import pandas as pd
import anndata as ad
import geopandas as gpd
from shapely.geometry import Point

import matplotlib.pyplot as plt
import math

In [ ]:
import matplotlib
matplotlib.rcParams['svg.fonttype'] = 'none'

# Import and preprocess

## Helper functions

In [ ]:
def guess_label_column(gdf: gpd.GeoDataFrame) -> str | None:
    """Pick a reasonable label column from common QuPath exports."""
    candidates = [
        "classification_name",  # from nested 'classification.name'
        "name",
        "label",
        "annotation",
        "class",
        "type",
    ]
    cols = [c for c in gdf.columns if c != gdf.geometry.name]
    for c in candidates:
        if c in cols:
            return c
    return cols[0] if cols else None


def ensure_label_column(gdf: gpd.GeoDataFrame, prefer: str | None) -> str | None:
    """Ensure there's a flat text label column to use."""
    if prefer and prefer in gdf.columns:
        return prefer

    # Some QuPath exports store nested dict in 'classification' with key 'name'
    if "classification" in gdf.columns and "classification_name" not in gdf.columns:
        def _extract_name(v):
            try:
                if isinstance(v, dict):
                    return v.get("name")
                if isinstance(v, str):
                    return json.loads(v).get("name")
            except Exception:
                pass
            return None
        gdf["classification_name"] = gdf["classification"].map(_extract_name)

    return guess_label_column(gdf)


def make_points_gdf(XY: np.ndarray, index, crs) -> gpd.GeoDataFrame:
    """Create a GeoDataFrame of points from (x, y) coordinates."""
    points = gpd.GeoSeries((Point(float(x), float(y)) for x, y in XY),
                           index=index, name="geometry", crs=crs)
    return gpd.GeoDataFrame(geometry=points)


def aggregate_labels(join_df: pd.DataFrame, label_col: str, obs_index) -> pd.Series:
    """Aggregate multiple matches per cell into a semicolon-separated label string."""
    if label_col not in join_df.columns:
        s = join_df.index.to_series().groupby(level=0).size()
        out = pd.Series(index=obs_index, dtype="object")
        out.loc[s.index] = "in_annotation"
        return out

    agg = (
        join_df[label_col]
        .groupby(level=0)
        .apply(lambda s: ";".join(sorted(str(v) for v in set(s.dropna()))))
    )
    out = pd.Series(index=obs_index, dtype="object")
    out.loc[agg.index] = agg
    return out


In [ ]:
def build_palettes_from_adata(adata, palette_specs):
    """
    Build labeled color palettes for categorical columns in adata.obs.

    Parameters
    ----------
    adata : AnnData
        Must have .obs DataFrame containing categorical columns.
    palette_specs : dict
        Mapping {column_name: palette} where palette can be:
          - a string palette name (e.g. "tab10")
          - a list of RGB colors (custom)

    Returns
    -------
    dict
        {column_name: {label: color}} mapping.
    """
    custom_palettes = {}

    for col, palette in palette_specs.items():
        if col not in adata.obs.columns:
            print(f"⚠️ Warning: '{col}' not found in adata.obs — skipping.")
            continue

        unique_vals = sorted(adata.obs[col].astype(str).dropna().unique())
        n_unique = len(unique_vals)

        # If user passed a name → generate via seaborn
        if isinstance(palette, str):
            pal_colors = sns.color_palette(palette, n_colors=n_unique)
        # If user passed a list → use directly
        elif isinstance(palette, (list, tuple)):
            pal_colors = palette[:n_unique]
        else:
            raise ValueError(f"Unsupported palette type for '{col}': {type(palette)}")

        color_dict = dict(zip(unique_vals, pal_colors))
        custom_palettes[col] = color_dict

    print(f"✅ Built palettes for {len(custom_palettes)} columns.")
    return custom_palettes


In [ ]:
def plot_celltype_spatial_single_split_legend(
    df,
    color_by="celltype",
    sample_id=None,
    title=None,
    palette_dict=None,         # ✅ added
    palette_name="tab20",
    s=1.5,
    save_svg=True,
    output_prefix="spatial_plot",
    legend_title=None,
):
    """
    Nature Genetics–style spatial scatterplot for one sample,
    saving main plot as PNG (raster) and legend separately as SVG (vector).
    """
    sns.set_style("white")
    sns.set_context("talk")

    # --- Subset one sample ---
    if sample_id is not None:
        df = df[df["sample_id"] == sample_id].copy()
        if df.empty:
            raise ValueError(f"Sample ID '{sample_id}' not found in DataFrame.")

    # --- Colors ---
    unique_labels = sorted(df[color_by].dropna().unique())
    if palette_dict is not None and color_by in palette_dict:
        color_dict = palette_dict[color_by]
    else:
        palette = sns.color_palette(palette_name, n_colors=len(unique_labels))
        color_dict = dict(zip(unique_labels, palette))

    # --- Main plot ---
    fig, ax = plt.subplots(figsize=(6, 5), dpi=300)
    sns.scatterplot(
        data=df,
        x="X_coord", y="Y_coord",
        hue=color_by, palette=color_dict,
        s=s, alpha=0.9, linewidth=0,
        rasterized=True, ax=ax, legend=False
    )
    ax.invert_yaxis(); ax.set_aspect("equal", adjustable="box")
    for spine in ["top", "right", "left", "bottom"]:
        ax.spines[spine].set_visible(False)
    ax.set_xticks([]); ax.set_yticks([])
    ax.set_xlabel(""); ax.set_ylabel("")
    plt.tight_layout()

    # --- Save main figure ---
    fname_main = f"{output_prefix}_{sample_id or 'sample'}_main.png"
    fig.savefig(fname_main, dpi=300, bbox_inches="tight", transparent=True, format="png")
    print(f"Saved main figure: {fname_main}")

    # --- Legend ---
    fig_leg, ax_leg = plt.subplots(figsize=(3, 0.5 * len(unique_labels)), dpi=300)
    handles = [
        plt.Line2D([0], [0], marker='o', color='none', label=label,
                   markerfacecolor=color_dict[label], markersize=8)
        for label in unique_labels
    ]
    ax_leg.legend(handles=handles, loc="center left", frameon=False,
                  title=legend_title or color_by, title_fontsize=14, fontsize=14)
    ax_leg.axis("off")
    plt.tight_layout()

    if save_svg:
        fname_leg = f"{output_prefix}_{sample_id or 'sample'}_legend.svg"
        fig_leg.savefig(fname_leg, dpi=300, bbox_inches="tight", transparent=True, format="svg")
        print(f"Saved legend: {fname_leg}")

    plt.close(fig); plt.close(fig_leg)


## Analysis

In [ ]:

# --- REQUIRED: set your paths ---
ADATA_PATH = '../../../Broad_SpatialFoundation/test_data/10X_VisiumHD_LUAD_FFPE/adata.h5ad'
GEOJSON_PATH = "../../../Broad_SpatialFoundation/test_data/10X_VisiumHD_LUAD_FFPE/Visium_HD_Human_Lung_Cancer_HD_Only_Experiment1_tissue_image.btf - 20x_BF_01.geojson"

# --- OPTIONAL: customize these ---
OBSM_KEY   = "spatial"                         # obsm key with (x, y) in H&E pixel space
OUT_PATH   = None                                 # None -> will save next to ADATA_PATH with .annotated.h5ad
OUT_COL    = "region_annotation"                  # adata.obs column to write labels into
LABEL_COL  = 'name'                                 # if you know your label field name in the GeoJSON, set it (e.g., "name")
                                                   # otherwise the notebook tries to guess it

In [ ]:
rawdata = sc.read_h5ad(ADATA_PATH)

In [ ]:
XY = np.asarray(rawdata.obsm[OBSM_KEY])

In [ ]:
ann = gpd.read_file(GEOJSON_PATH)
if ann.empty:
    raise ValueError("Annotation GeoJSON has no geometries.")

ann = ann[ann.geometry.notnull()].copy()
ann = ann[ann.geometry.geom_type.isin(["Polygon", "MultiPolygon"])].copy()
if ann.empty:
    raise ValueError("No Polygon/MultiPolygon geometries found in the GeoJSON.")

label_col = ensure_label_column(ann, LABEL_COL)
if label_col is None:
    label_col = "_layer"
    ann[label_col] = "annotation"

print("Annotation label column:", label_col)
display(ann.head())


In [ ]:
# Make sure CRS aligns; QuPath pixel coords commonly have no CRS, which is fine
pts = make_points_gdf(XY, rawdata.obs_names, crs=ann.crs)

# Join with predicate='within' (Shapely 2+ / GeoPandas >=0.10)
joined = gpd.sjoin(pts, ann[[label_col, ann.geometry.name]], how="left", predicate="within")

# Aggregate overlaps and fill missing
labels = aggregate_labels(joined, label_col, obs_index=rawdata.obs_names).fillna("unlabeled")
rawdata.obs[OUT_COL] = pd.Categorical(labels)

print("Done assigning labels. Preview value counts:")
display(rawdata.obs[OUT_COL].value_counts(dropna=False).to_frame("count"))


In [ ]:
rawdata.obs[OUT_COL] = rawdata.obs[OUT_COL].replace({'': 'Unassigned', 'Grade 2 Tumor;Grade 3 Tumor': 'Unassigned',})

In [ ]:
rawdata.obs = pd.concat([rawdata.obs, pd.DataFrame(rawdata.obsm['spatial'], index=rawdata.obs_names, columns=['X_coord','Y_coord'])],axis=1)

In [ ]:
region_df = rawdata.obs[['region_annotation', 'X_coord','Y_coord']]

region_df['sample_id'] = 'VisiumHD_LUAD'

In [ ]:
tab_filtered = sns.color_palette()
tab_filtered = [c for i,c in enumerate(tab_filtered) if i not in [4,6]]

tab20_filtered = sns.color_palette('tab20') + sns.color_palette('tab20c')[:11]
tab20_filtered = [c for i,c in enumerate(tab20_filtered) if i not in [8,9,12,13]]

In [ ]:
palette_specs = {
            "region_annotation": tab_filtered,
        }

palette_dict_1 = build_palettes_from_adata(rawdata, palette_specs)

In [ ]:
plot_celltype_spatial_single_split_legend(
    region_df,
    color_by="region_annotation",
    sample_id=None,
    title='LUAD\nPathologist-annotated region',
    palette_dict=palette_dict_1,
    s=1.5,
    save_svg=True,
    output_prefix="../../../SpatialFusion/results/figures_Fig4/LUAD_pathregion",
    legend_title='Path.-annotated region'
)

# Embed scGPT

In [ ]:
import os
import sys
import json
import logging
import warnings
import argparse
from typing import List, Optional
from pathlib import Path

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# --- Logging ---
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s] %(levelname)s - %(message)s",
    datefmt="%H:%M:%S",
)
log = logging.getLogger("hest")

import pandas as pd
import matplotlib.pyplot as plt

from huggingface_hub import login as hf_login
import datasets

import numpy as np
import scanpy as sc
import tifffile
import shapely.wkb  # access as shapely.wkb.loads(...)
from PIL import Image
from tqdm import tqdm

import torch
from torchvision import transforms
import timm

In [ ]:
def run_embed_scGPT(
    dataset_path: Path,
    model_dir: Path,  # dir containing best_model.pt, args.json, vocab.json
    output_dir: Path,
    n_hvg: int = 1200,
    gene_col: str = "index",
    layer_key: str = "X",
    log_norm: bool = False,
    seed: int = 42,
    max_seq_len: int = 1200,
    batch_size: int = 16,
    input_bins: int = 51,
    model_run: str = "pretrained",
    num_workers: int = 0,
):
    # Lazy imports from user repo
    sys.path.append(str(model_dir.parent))  # allow e.g., /.../Broad_SpatialFoundation/
    try:
        from sc_foundation_evals import cell_embeddings, scgpt_forward, data, model_output  # noqa: F401
        from sc_foundation_evals.helpers.custom_logging import log as sc_log  # noqa: F401
    except Exception as e:
        raise ImportError(
            "Could not import sc_foundation_evals from your codebase. "
            "Ensure the repository is available and model_dir parent is in sys.path."
        ) from e

    # Create model
    scgpt_model = scgpt_forward.scGPT_instance(
        saved_model_path=str(model_dir),
        model_run=model_run,
        batch_size=batch_size,
        save_dir=str(output_dir),
        num_workers=num_workers,
        explicit_save_dir=True,
    )

    # Configs and weights
    scgpt_model.create_configs(seed=seed, max_seq_len=max_seq_len, n_bins=input_bins)
    scgpt_model.load_pretrained_model()

    # Data prep
    input_data = data.InputData(adata_dataset_path=str(dataset_path))
    vocab_list = scgpt_model.vocab.get_stoi().keys()

    input_data.preprocess_data(
        gene_vocab=vocab_list,
        model_type="scGPT",
        gene_col=gene_col,
        data_is_raw=not log_norm,
        counts_layer=layer_key,
        n_bins=input_bins,
        n_hvg=n_hvg,
    )

    scgpt_model.tokenize_data(
        data=input_data, input_layer_key="X_binned", include_zero_genes=False
    )

    scgpt_model.extract_embeddings(data=input_data)

    out_parquet = output_dir / "scGPT.parquet"
    pd.DataFrame(
        input_data.adata.obsm["X_scGPT"],
        index=input_data.adata.obs["cell_id"] if "cell_id" in input_data.adata.obs.columns else input_data.adata.obs.index,
    ).to_parquet(out_parquet)
    print(f"[scGPT] Wrote embeddings: {out_parquet}")

In [ ]:
os.makedirs(pl.Path(ADATA_PATH).parent / 'embeddings', exist_ok=True)

In [ ]:
run_embed_scGPT(
    dataset_path=ADATA_PATH,
    model_dir=pl.Path('../../../Broad_SpatialFoundation/scGPT_model/'),
    output_dir=pl.Path(ADATA_PATH).parent / 'embeddings',
    n_hvg=1200,
    gene_col='index',
    layer_key='X',
    log_norm=False,
    seed=42,
    max_seq_len=1200,
    batch_size=16,
    input_bins=51,
    model_run="pretrained",
    num_workers=0,
)

# Embed UNI

In [ ]:
def load_UNI_model(model_path: str, device: str = "cuda"):
    timm_kwargs = {
        'model_name': 'vit_giant_patch14_224',
        'img_size': 224,
        'patch_size': 14,
        'depth': 24,
        'num_heads': 24,
        'init_values': 1e-5,
        'embed_dim': 1536,
        'mlp_ratio': 2.66667 * 2,
        'num_classes': 0,
        'no_embed_class': True,
        'mlp_layer': timm.layers.SwiGLUPacked,
        'act_layer': torch.nn.SiLU,
        'reg_tokens': 8,
        'dynamic_img_size': True
    }

    model = timm.create_model(pretrained=False, **timm_kwargs)
    model.load_state_dict(torch.load(model_path, map_location="cpu"), strict=True)
    model.eval().to(device)

    transform = transforms.Compose([
        transforms.Lambda(lambda im: im.convert('RGB')),  # ensure 3 channels
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406),
                             std=(0.229, 0.224, 0.225)),
    ])
    return model, transform


def process_HDsample_and_embed_UNI(
    adata, wsi_path, sample_id, model_path, out_root="VisiumHD_processed_data", device="cuda"
):
    print('Load UNI model')
    model, transform = load_UNI_model(model_path, device)

    # ensure output dirs
    os.makedirs(f"{out_root}/{sample_id}/embeddings", exist_ok=True)

    with tifffile.TiffFile(str(wsi_path)) as tif:
        wsi = tif.pages[0].asarray()

    he_coords = adata.obsm['spatial']

    batch_size = 128
    embeddings = []
    cell_ids = []
    batch_imgs, batch_ids = [], []

    print(f"Embedding {len(he_coords)} image patches in batches of {batch_size}...")
    for cid, (x, y) in tqdm(zip(adata.obs_names, he_coords), total=len(adata)):
        x, y = int(x), int(y)
        x0, x1 = x - 128, x + 128
        y0, y1 = y - 128, y + 128

        pad_x0 = max(0, -x0)
        pad_x1 = max(0, x1 - wsi.shape[1])
        pad_y0 = max(0, -y0)
        pad_y1 = max(0, y1 - wsi.shape[0])

        # slice & pad (handles edges)
        patch = np.pad(
            wsi[max(0, y0):min(wsi.shape[0], y1), max(0, x0):min(wsi.shape[1], x1)],
            ((pad_y0, pad_y1), (pad_x0, pad_x1)) + ((0, 0),) if wsi.ndim == 3 else ((0,0),),
            mode="constant"
        )

        # ensure HxW or HxWxC with C>=1
        if patch.ndim == 2:
            patch = np.stack([patch]*3, axis=-1)  # grayscale -> RGB

        if patch.shape[:2] != (256, 256):
            continue

        # drop alpha or extra channels if present (e.g., RGBA)
        if patch.shape[2] > 3:
            patch = patch[:, :, :3]

        img = Image.fromarray(patch).convert('RGB')
        tensor_img = transform(img)

        batch_imgs.append(tensor_img)
        batch_ids.append(cid)

        if len(batch_imgs) == batch_size:
            img_tensor = torch.stack(batch_imgs).to(device)
            with torch.inference_mode():
                if device.startswith('cuda'):
                    with torch.autocast(device_type='cuda', dtype=torch.float16):
                        batch_embs = model(img_tensor).to(torch.float16).cpu().numpy()
                else:
                    batch_embs = model(img_tensor).cpu().numpy()

            embeddings.extend(batch_embs)
            cell_ids.extend(batch_ids)
            batch_imgs.clear()
            batch_ids.clear()

    # Final batch
    if batch_imgs:
        img_tensor = torch.stack(batch_imgs).to(device)
        with torch.inference_mode():
            if device.startswith('cuda'):
                with torch.autocast(device_type='cuda', dtype=torch.float16):
                    batch_embs = model(img_tensor).to(torch.float16).cpu().numpy()
            else:
                batch_embs = model(img_tensor).cpu().numpy()
        embeddings.extend(batch_embs)
        cell_ids.extend(batch_ids)

    # Save
    df = pd.DataFrame(embeddings, index=cell_ids)
    out_path = f"{out_root}/{sample_id}/embeddings/UNI.parquet"
    df.to_parquet(out_path, index=True)
    print(f"Saved {len(df)} embeddings to {out_path}")


In [ ]:
device = "cuda:5" if torch.cuda.is_available() else "cpu"  # target GPU 5 explicitly

base_dir = Path('../../../Broad_SpatialFoundation/test_data/')
raw_dir  = Path('../../../Broad_SpatialFoundation/test_data/')
sample_id = '10X_VisiumHD_LUAD_FFPE'

print(f"\n🔄 Processing {sample_id}")
output_path = base_dir / sample_id
img_dir    = raw_dir / sample_id

adata = sc.read_h5ad(output_path / 'adata.h5ad')

process_HDsample_and_embed_UNI(
    adata,
    img_dir / 'Visium_HD_Human_Lung_Cancer_HD_Only_Experiment1_tissue_image.btf',           
    sample_id,
    "../../../Broad_SpatialFoundation/UNI/pytorch_model.bin",
    out_root=str(base_dir),
    device=device
)


# Embed sample

In [ ]:
from spatialfusion.embed.embed import AEInputs, run_full_embedding

In [ ]:
basepath = pl.Path('../../../Broad_SpatialFoundation/test_data/')
sample_name = '10X_VisiumHD_LUAD_FFPE'
output_dir = basepath / sample_name

uni_df = pd.read_parquet(pl.Path(output_dir) / 'embeddings' / 'UNI.parquet')
scgpt_df = pd.read_parquet(pl.Path(output_dir) / 'embeddings' / 'scGPT.parquet')

adata = sc.read_h5ad(basepath / sample_name / 'adata.h5ad')
adata.obs = pd.concat([adata.obs, pd.DataFrame(adata.obsm['spatial'], index=adata.obs_names, columns=['X_coord','Y_coord'])],axis=1)
adata.obs["sample_id"] = sample_name

ae_inputs_by_sample = {
    sample_name: AEInputs(adata=adata, z_uni=uni_df, z_scgpt=scgpt_df),
}

## Embeddings average

In [ ]:
# this uses the average version
embeddings_df = run_full_embedding(
    ae_inputs_by_sample=ae_inputs_by_sample,
    ae_model_path='../../../Broad_SpatialFoundation/checkpoint_dir_ae/paired_model_6c22d731.pt',
    gcn_model_path='../../../Broad_SpatialFoundation/checkpoint_dir_gcn/gcn_20250828-123835_e926ee8d/model.pt',
    device="cuda:0",
    combine_mode="average",
    spatial_key='spatial',
    celltype_key=None,
    save_ae_dir=None,  # optional
)

In [ ]:
embeddings_df.index = embeddings_df.cell_id

out_path = pl.Path(output_dir / "embeddings" / "NicheFinder_new.parquet")
embeddings_df.to_parquet(out_path)

## Embeddings H&E only

In [ ]:
# this uses the average version
embeddings_df = run_full_embedding(
    ae_inputs_by_sample=ae_inputs_by_sample,
    ae_model_path='../../../Broad_SpatialFoundation/checkpoint_dir_ae/paired_model_6c22d731.pt',
    gcn_model_path='../../../Broad_SpatialFoundation/checkpoint_dir_gcn/gcn_20251001-102239_fa6fe395/model.pt',
    device="cuda:0",
    combine_mode="z1",
    spatial_key='spatial',
    celltype_key=None,
    save_ae_dir=None,  # optional
)

In [ ]:
embeddings_df.index = embeddings_df.cell_id

out_path = pl.Path(output_dir / "embeddings"/ "NicheFinder_he_new.parquet")
embeddings_df.to_parquet(out_path)

# Finetune model

First here I need to compute the decoupler estimated pathways.

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
import warnings
import pathlib as pl
from scipy.sparse import issparse
import warnings

import scanpy as sc
import decoupler as dc

warnings.filterwarnings("ignore", category=FutureWarning)
sc.set_figure_params(figsize=(3, 3), frameon=False)


In [ ]:
progeny = dc.op.progeny(organism="human")
progeny

In [ ]:
def get_pathway_score(adata, spatial_key="spatial_he", bw=100, cutoff=0.1, net=progeny):
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    
    adata.layers["norm"] = adata.X.copy()
    dc.pp.knn(adata, key=spatial_key, bw=bw, cutoff=cutoff)
    adata.X = adata.obsp[f"{spatial_key}_connectivities"].dot(adata.X)

    dc.mt.ulm(data=adata, net=net)
    score = dc.pp.get_obsm(adata=adata, key="score_ulm")
    return adata, score

In [ ]:
actionable_pathways = ['Androgen', 'EGFR', 'Estrogen', 'JAK-STAT', 'MAPK', 'NFkB',
       'PI3K', 'TGFb', 'TNFa', 'VEGF',]

In [ ]:
_, pathway_score = get_pathway_score(adata, spatial_key="spatial", bw=100, cutoff=0.1)
pathway_score = pathway_score.obsm['score_ulm'].loc[:,actionable_pathways]

Now I can finetune.

In [ ]:
from spatialfusion.finetune.finetune import finetune_models

In [ ]:
adata = sc.read_h5ad(basepath / sample_name / 'adata.h5ad')
adata.obs = pd.concat([adata.obs, pd.DataFrame(adata.obsm['spatial'], index=adata.obs_names, columns=['X_coord','Y_coord'])],axis=1)
adata.obs["sample_id"] = sample_name

In [ ]:
adatas = {sample_name: adata}

preloaded_data = {
    sample_name: (uni_df.loc[adata.obs_names], scgpt_df.loc[adata.obs_names])
}

preloaded_pathway_data = {
    sample_name: pathway_score.loc[adata.obs_names]
}

In [ ]:
finetune_models(
    samples=[sample_name],
    pretrained_ae='../../../Broad_SpatialFoundation/checkpoint_dir_ae/paired_model_6c22d731.pt',
    pretrained_gcn='../../../Broad_SpatialFoundation/checkpoint_dir_gcn/gcn_20250828-123835_e926ee8d/model.pt',
    save_dir='../../../Broad_SpatialFoundation/finetuned_LUAD_hd',
    preloaded_data=preloaded_data,   # instead of base_path
    adatas=adatas,                   # instead of loading from disk
    preloaded_pathway_data=preloaded_pathway_data,
    ae_epochs=5,
    gcn_epochs=10,
    ae_batch_size=128,
    gcn_batch_size=2,
    latent_dim=64,
    knn_k=30,
    subgraph_size=5000,
    stride=2500,
    use_cls_loss=True,
    spatial_key='spatial',
)


## Now embed finetuned

In [ ]:
basepath = pl.Path('../../../Broad_SpatialFoundation/test_data/')
sample_name = '10X_VisiumHD_LUAD_FFPE'
output_dir = basepath / sample_name

In [ ]:
uni_df = pd.read_parquet(pl.Path(output_dir) / 'embeddings' / 'UNI.parquet')
scgpt_df = pd.read_parquet(pl.Path(output_dir) / 'embeddings' / 'scGPT.parquet')

In [ ]:
adata = sc.read_h5ad(basepath / sample_name / 'adata.h5ad')
adata.obs = pd.concat([adata.obs, pd.DataFrame(adata.obsm['spatial'], index=adata.obs_names, columns=['X_coord','Y_coord'])],axis=1)
adata.obs["sample_id"] = sample_name

In [ ]:
ae_inputs_by_sample = {
    sample_name: AEInputs(adata=adata, z_uni=uni_df, z_scgpt=scgpt_df),
}

In [ ]:
# this uses the average version
embeddings_df = run_full_embedding(
    ae_inputs_by_sample=ae_inputs_by_sample,
    ae_model_path='../../../Broad_SpatialFoundation/finetuned_LUAD_hd/paired_model_finetuned.pt',
    gcn_model_path='../../../Broad_SpatialFoundation/finetuned_LUAD_hd/gcn_finetuned/model.pt',
    device="cuda:0",
    combine_mode="average",
    spatial_key='spatial',
    celltype_key=None,
    save_ae_dir=None,  # optional
)

In [ ]:
embeddings_df.index = embeddings_df.cell_id

out_path = pl.Path(output_dir / "embeddings"/ "NicheFinder_finetuned_new.parquet")
embeddings_df.to_parquet(out_path)

# SDMBench

## Helper functions

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import scanpy as sc

from scipy.spatial import distance
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *


In [ ]:
def compute_PAS_fast(clusterlabel, location, k=10):
    clusterlabel = np.array(clusterlabel)
    location = np.array(location)

    # Fit NearestNeighbors (ignore self-match later)
    nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='auto').fit(location)
    distances, indices = nbrs.kneighbors(location)

    # Remove self (first column is self in most cases)
    neighbor_indices = indices[:, 1:]  # shape: (n_samples, k)

    # Check PAS condition
    mismatches = np.array([
        np.sum(clusterlabel[neighbor_indices[i]] != clusterlabel[i]) > (k / 2)
        for i in range(len(clusterlabel))
    ])

    return np.sum(mismatches) / len(clusterlabel)


def compute_CHAOS_fast(clusterlabel, location):
    clusterlabel = np.array(clusterlabel)
    location = np.array(location)
    matched_location = StandardScaler().fit_transform(location)

    clusterlabel_unique = np.unique(clusterlabel)
    dist_val = 0
    total_count = 0

    for k in tqdm(clusterlabel_unique, desc="Computing CHAOS"):
        cluster_mask = clusterlabel == k
        location_cluster = matched_location[cluster_mask]
        n = location_cluster.shape[0]

        if n <= 2:
            continue

        # Use NearestNeighbors to find 1-NN distances
        nbrs = NearestNeighbors(n_neighbors=2, algorithm='auto').fit(location_cluster)
        distances, _ = nbrs.kneighbors(location_cluster)

        # distances[:, 0] is zero (self), distances[:, 1] is nearest neighbor
        dist_val += np.sum(distances[:, 1])
        total_count += n

    return dist_val / total_count if total_count > 0 else np.nan


def compute_ASW_fast(adata, pred_key, spatial_key='spatial'):
    coords = adata.obsm[spatial_key]
    labels = adata.obs[pred_key]
    return silhouette_score(X=coords, labels=labels, metric='euclidean')

def compute_ARI(adata,gt_key,pred_key):
        return adjusted_rand_score(adata.obs[gt_key],adata.obs[pred_key])

def compute_NMI(adata,gt_key,pred_key):
    return normalized_mutual_info_score(adata.obs[gt_key],adata.obs[pred_key])

def compute_HOM(adata,gt_key,pred_key):
    return homogeneity_score(adata.obs[gt_key],adata.obs[pred_key])

def compute_COM(adata,gt_key,pred_key):
    return completeness_score(adata.obs[gt_key],adata.obs[pred_key])

In [ ]:
basepath = pl.Path('../../../Broad_SpatialFoundation/test_data/')
sample_name = '10X_VisiumHD_LUAD_FFPE'
output_dir = basepath / sample_name

In [ ]:
adata = rawdata.copy()

emb_df = pd.read_parquet(basepath / sample_name / 'embeddings' / 'NicheFinder_new.parquet')

emb_he_df = pd.read_parquet(basepath / sample_name / 'embeddings' / 'NicheFinder_he_new.parquet')

emb_finetuned_df = pd.read_parquet(basepath / sample_name / 'embeddings' / 'NicheFinder_finetuned_new.parquet')

nichecompass_df = pd.read_parquet(basepath / sample_name / 'embeddings' / 'nichecompass.parquet')

banksy_df = pd.read_parquet(basepath / sample_name / 'embeddings' / 'banksy.parquet')

banksy_08_df = pd.read_parquet(basepath / sample_name / 'embeddings' / 'banksy_08.parquet')

scgptspatial_df = pd.read_parquet(basepath / sample_name / 'embeddings' / 'scGPTspatial.parquet')

omiclip_text_df = pd.read_parquet(basepath / sample_name / 'embeddings' / 'OmiCLIP_text_emb.parquet')

omiclip_image_df = pd.read_parquet(basepath / sample_name / 'embeddings' / 'OmiCLIP_image_emb.parquet')

In [ ]:
adata = adata[adata.obs_names.intersection(emb_df.index)].copy()

adata.obsm['gcn'] = emb_df.loc[adata.obs_names,['0','1','2','3','4','5','6','7','8','9']]

adata.obsm['gcn_he'] = emb_he_df.loc[adata.obs_names,['0','1','2','3','4','5','6','7','8','9']]

adata.obsm['gcn_finetuned'] = emb_finetuned_df.loc[adata.obs_names,['0','1','2','3','4','5','6','7','8','9']]

adata.obsm['nichecompass'] = nichecompass_df.loc[adata.obs_names]

adata.obsm['banksy'] = banksy_df.loc[adata.obs_names]

adata.obsm['banksy_08'] = banksy_08_df.loc[adata.obs_names]

adata.obsm['scgptspatial'] = scgptspatial_df.loc[adata.obs_names]

adata.obsm['scgptspatial'] = scgptspatial_df.loc[adata.obs_names]

adata.obsm['omiclip_text'] = omiclip_text_df.loc[adata.obs_names]

adata.obsm['omiclip_image'] = omiclip_image_df.loc[adata.obs_names]

In [ ]:
adata_obs = pd.read_csv('benchmark_luad_hd_obs.csv', index_col=0)

adata_obs.index = adata_obs.index.astype(str)

adata.obs = adata_obs

In [ ]:
sc.pp.neighbors(adata, use_rep = 'gcn')

In [ ]:
sc.tl.leiden(adata, resolution=0.12, flavor="igraph", n_iterations=2)

In [ ]:
# Get cluster sizes
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)

# Make mapping: old → new (ranked by size)
mapping = {old: str(new) for new, old in enumerate(counts.index)}

# Apply mapping
adata.obs['leiden'] = adata.obs['leiden'].map(mapping).astype('category')

# (optional) sort categories by their new numeric label
adata.obs['leiden'].cat.reorder_categories(sorted(adata.obs['leiden'].cat.categories, key=int))

print("Cluster relabeling done ✅")
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)
display(counts)

In [ ]:
adata.obs['leiden_gcn'] =  adata.obs.leiden.replace({'11': '10', '12': '10', '13': '10', '14': '10',})

In [ ]:
sc.pp.neighbors(adata, use_rep = 'gcn_he')

In [ ]:
sc.tl.leiden(adata, resolution=0.1, flavor="igraph", n_iterations=2)

In [ ]:
# Get cluster sizes
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)

# Make mapping: old → new (ranked by size)
mapping = {old: str(new) for new, old in enumerate(counts.index)}

# Apply mapping
adata.obs['leiden'] = adata.obs['leiden'].map(mapping).astype('category')

# (optional) sort categories by their new numeric label
adata.obs['leiden'].cat.reorder_categories(sorted(adata.obs['leiden'].cat.categories, key=int))

print("Cluster relabeling done ✅")
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)
display(counts)

In [ ]:
adata.obs['leiden_gcn_he'] =  adata.obs.leiden.replace({'11': '10', '12': '10', '13': '10',})

In [ ]:
sc.pp.neighbors(adata, use_rep = 'gcn_finetuned')

In [ ]:
sc.tl.leiden(adata, resolution=0.14, flavor="igraph", n_iterations=2)

In [ ]:
# Get cluster sizes
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)

# Make mapping: old → new (ranked by size)
mapping = {old: str(new) for new, old in enumerate(counts.index)}

# Apply mapping
adata.obs['leiden'] = adata.obs['leiden'].map(mapping).astype('category')

# (optional) sort categories by their new numeric label
adata.obs['leiden'].cat.reorder_categories(sorted(adata.obs['leiden'].cat.categories, key=int))

print("Cluster relabeling done ✅")
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)
display(counts)

In [ ]:
adata.obs['leiden_gcn_finetuned'] =  adata.obs.leiden.replace({'11': '10', '12': '10',}) 

In [ ]:
sc.pp.neighbors(adata, use_rep = 'nichecompass')

In [ ]:
sc.tl.leiden(adata, resolution=0.4, flavor="igraph", n_iterations=2)

In [ ]:
# Get cluster sizes
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)

# Make mapping: old → new (ranked by size)
mapping = {old: str(new) for new, old in enumerate(counts.index)}

# Apply mapping
adata.obs['leiden'] = adata.obs['leiden'].map(mapping).astype('category')

# (optional) sort categories by their new numeric label
adata.obs['leiden'].cat.reorder_categories(sorted(adata.obs['leiden'].cat.categories, key=int))

print("Cluster relabeling done ✅")
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)
display(counts)

In [ ]:
adata.obs['leiden_nichecompass'] =  adata.obs.leiden

In [ ]:
sc.pp.neighbors(adata, use_rep = 'banksy')

In [ ]:
sc.tl.leiden(adata, resolution=0.2, flavor="igraph", n_iterations=2)

In [ ]:
# Get cluster sizes
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)

# Make mapping: old → new (ranked by size)
mapping = {old: str(new) for new, old in enumerate(counts.index)}

# Apply mapping
adata.obs['leiden'] = adata.obs['leiden'].map(mapping).astype('category')

# (optional) sort categories by their new numeric label
adata.obs['leiden'].cat.reorder_categories(sorted(adata.obs['leiden'].cat.categories, key=int))

print("Cluster relabeling done ✅")
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)
display(counts)

In [ ]:
adata.obs['leiden_banksy'] =  adata.obs.leiden.replace({'11': '10',})

In [ ]:
sc.pp.neighbors(adata, use_rep = 'banksy_08')

In [ ]:
sc.tl.leiden(adata, resolution=0.22, flavor="igraph", n_iterations=2)

In [ ]:
# Get cluster sizes
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)

# Make mapping: old → new (ranked by size)
mapping = {old: str(new) for new, old in enumerate(counts.index)}

# Apply mapping
adata.obs['leiden'] = adata.obs['leiden'].map(mapping).astype('category')

# (optional) sort categories by their new numeric label
adata.obs['leiden'].cat.reorder_categories(sorted(adata.obs['leiden'].cat.categories, key=int))

print("Cluster relabeling done ✅")
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)
display(counts)

In [ ]:
adata.obs['leiden_banksy_08'] =  adata.obs.leiden.replace({'11': '10', '12': '10','13': '10', '14': '10','15': '10',})

In [ ]:
sc.pp.neighbors(adata, use_rep = 'scgptspatial')

In [ ]:
sc.tl.leiden(adata, resolution=0.7, flavor="igraph", n_iterations=2)

In [ ]:
# Get cluster sizes
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)

# Make mapping: old → new (ranked by size)
mapping = {old: str(new) for new, old in enumerate(counts.index)}

# Apply mapping
adata.obs['leiden'] = adata.obs['leiden'].map(mapping).astype('category')

# (optional) sort categories by their new numeric label
adata.obs['leiden'].cat.reorder_categories(sorted(adata.obs['leiden'].cat.categories, key=int))

print("Cluster relabeling done ✅")
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)
display(counts)

In [ ]:
adata.obs['leiden_scgptspatial'] =  adata.obs.leiden

In [ ]:
sc.pp.neighbors(adata, use_rep = 'omiclip_text')

In [ ]:
sc.tl.leiden(adata, resolution=0.65, flavor="igraph", n_iterations=2)

In [ ]:
# Get cluster sizes
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)

# Make mapping: old → new (ranked by size)
mapping = {old: str(new) for new, old in enumerate(counts.index)}

# Apply mapping
adata.obs['leiden'] = adata.obs['leiden'].map(mapping).astype('category')

# (optional) sort categories by their new numeric label
adata.obs['leiden'].cat.reorder_categories(sorted(adata.obs['leiden'].cat.categories, key=int))

print("Cluster relabeling done ✅")
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)
display(counts)

In [ ]:
adata.obs['leiden_omiclip_text'] =  adata.obs.leiden.replace({'11': '10'})

In [ ]:
sc.pp.neighbors(adata, use_rep = 'omiclip_image')

In [ ]:
sc.tl.leiden(adata, resolution=0.1, flavor="igraph", n_iterations=2)

In [ ]:
# Get cluster sizes
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)

# Make mapping: old → new (ranked by size)
mapping = {old: str(new) for new, old in enumerate(counts.index)}

# Apply mapping
adata.obs['leiden'] = adata.obs['leiden'].map(mapping).astype('category')

# (optional) sort categories by their new numeric label
adata.obs['leiden'].cat.reorder_categories(sorted(adata.obs['leiden'].cat.categories, key=int))

print("Cluster relabeling done ✅")
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)
display(counts)

In [ ]:
adata.obs['leiden_omiclip_image'] =  adata.obs.leiden.replace({'11': '10'})

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

sc.tl.pca(adata)

sc.pp.neighbors(adata, use_rep='X_pca')

In [ ]:
sc.tl.leiden(adata, resolution=0.8, flavor="igraph", n_iterations=2)

In [ ]:
# Get cluster sizes
counts = adata.obs['leiden'].value_counts().sort_values(ascending=False)

# Make mapping: old → new (ranked by size)
mapping = {old: str(new) for new, old in enumerate(counts.index)}

# Apply mapping
adata.obs['leiden'] = adata.obs['leiden'].map(mapping).astype('category')

# (optional) sort categories by their new numeric label
adata.obs['leiden'].cat.reorder_categories(sorted(adata.obs['leiden'].cat.categories, key=int))

print("Cluster relabeling done ✅")
display(counts)

In [ ]:
adata.obs['leiden_scanpy'] = adata.obs.leiden.replace({'11': '10',})

In [ ]:
adata.obs.to_csv('benchmark_luad_hd_obs.csv')

## Compute metrics

In [ ]:
def compute_all_metrics(adata, clustering_keys, ground_truth_key='path_region', spatial_key='spatial_px'):
    results = {}

    for method_name, cluster_key in clustering_keys.items():
        metrics = {
            'ARI': compute_ARI(adata, cluster_key, ground_truth_key),
            'NMI': compute_NMI(adata, cluster_key, ground_truth_key),
            'HOM': compute_HOM(adata, cluster_key, ground_truth_key),
            'COM': compute_COM(adata, cluster_key, ground_truth_key),
            'PAS': compute_PAS_fast(adata.obs[cluster_key], adata.obsm[spatial_key]),
            'CHAOS': compute_CHAOS_fast(adata.obs[cluster_key], adata.obsm[spatial_key]),
        }
        results[method_name] = metrics

    return pd.DataFrame(results)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap

def format_number(value):
    """Format numbers: scientific notation if <0.01, else 2 decimals."""
    if pd.isna(value):
        return ""
    if abs(value) < 0.01 and value != 0:
        return f"{value:.0e}"  # 1 decimal in scientific notation, e.g. 3.4e-04
    else:
        return f"{value:.2f}"  # two decimals otherwise

def plot_benchmark_heatmap(
    results_df,
    title="Spatial clustering benchmark",
    savefig=None,
    metric_order=None,
):
    """
    Nature Genetics–style benchmarking heatmap showing method rankings across metrics.
    Allows manual control of metric order.
    """

    lower_better = {'PAS', 'CHAOS'}

    # --- Default metric order ---
    if metric_order is None:
        metric_order = list(results_df.index)

    # --- Normalize scores ---
    df_norm = results_df.copy()
    for metric in df_norm.index:
        vals = df_norm.loc[metric]
        if metric in lower_better:
            vals = -vals
        df_norm.loc[metric] = (vals - vals.min()) / (vals.max() - vals.min() + 1e-9)

    # --- Rank per metric ---
    ranks = results_df.copy()
    for metric in ranks.index:
        ranks.loc[metric] = results_df.loc[metric].rank(ascending=(metric in lower_better))

    # --- Prepare longform for plotting ---
    df_plot = df_norm.reset_index().melt(
        id_vars='index', var_name='Method', value_name='Normalized'
    ).rename(columns={'index': 'Metric'})

    df_plot['Raw'] = results_df.reset_index().melt(
        id_vars='index', var_name='Method', value_name='Raw'
    )['Raw']

    df_plot['Rank'] = ranks.reset_index().melt(
        id_vars='index', var_name='Method', value_name='Rank'
    )['Rank']

    # Add directional arrows
    df_plot['MetricLabel'] = df_plot['Metric'].apply(
        lambda m: f"{m} {'↓' if m in lower_better else '↑'}"
    )

    # --- Construct ordered MetricLabel list ---
    metric_order_labels = []
    for m in metric_order:
        arrow = '↓' if m in lower_better else '↑'
        metric_order_labels.append(f"{m} {arrow}")

    # --- Heatmap data matrix ---
    method_order = results_df.columns.tolist()
    df_matrix = df_plot.pivot_table(
        index="MetricLabel", columns="Method", values="Normalized"
    ).loc[metric_order_labels, method_order]

    # --- Aesthetics ---
    sns.set_theme(style="white", context="talk")

    fig, ax = plt.subplots(figsize=(1.3 * len(method_order), 0.8 * len(metric_order)), dpi=300)
    # Enhance contrast near the top (gamma correction)
    gamma = 3  ### THIS IS ONLY FOR THE COLOR FOR PLOTTING PURPOSES, NOT THE NUMBERS!
    df_matrix_contrast = df_matrix ** gamma
    sns.heatmap(
        df_matrix_contrast,
        #cmap="vlag",
        cmap = LinearSegmentedColormap.from_list(
            "vlag_red",
            ["#fee8ef",  # very light pink
             "#f4a3a8",  # pastel red
             "#d95858",  # mid red
             "#b40426"]  # vlag red (vivid crimson)
        ),
        cbar=False,
        ax=ax,
        linewidths=0,
        square=True,
    )

    # --- Adaptive text color (white on dark, black on light) ---
    #cmap = plt.get_cmap("vlag")
    cmap = LinearSegmentedColormap.from_list(
        "vlag_red",
        ["#fee8ef",  # very light pink
         "#f4a3a8",  # pastel red
         "#d95858",  # mid red
         "#b40426"]  # vlag red (vivid crimson)
    )

    for i, metric in enumerate(df_matrix.index):
        base_metric = metric.split()[0]
        for j, method in enumerate(df_matrix.columns):
            raw_val = results_df.loc[base_metric, method]
            norm_val = df_matrix.loc[metric, method]

            # Compute luminance for adaptive color
            rgb = np.array(cmap(norm_val)[:3])
            luminance = 0.2126 * rgb[0] + 0.7152 * rgb[1] + 0.0722 * rgb[2]
            text_color = "black" if luminance > 0.5 else "white"

            ax.text(
                j + 0.5, i + 0.5,
                format_number(raw_val),
                ha='center', va='center',
                color=text_color,
                fontsize=8,
                fontweight='normal',
            )

    # --- Formatting ---
    ax.set_title(title, fontsize=10, pad=14, fontweight='normal')
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right", fontsize=10, fontweight='normal')
    ax.set_yticklabels(ax.get_yticklabels(), fontsize=10, fontweight='normal')

    for spine in ax.spines.values():
        spine.set_visible(False)

    plt.tight_layout()

    if savefig:
        fig.savefig(
            savefig,
            bbox_inches="tight",
            dpi=300,
            format=savefig.split('.')[-1],
            transparent=True
        )
        print(f"Saved: {savefig}")

    plt.show()



In [ ]:
adata_obs = pd.read_csv('benchmark_luad_hd_obs.csv', index_col=0)

adata_obs.index = adata_obs.index.astype(str)

adata.obs = adata_obs

In [ ]:
clustering_keys = {
    'SpatialFusion': 'leiden_gcn',
    'SpatialFusion (H&E)': 'leiden_gcn_he',
    'SpatialFusion (finetuned)': 'leiden_gcn_finetuned',
    'NicheCompass': 'leiden_nichecompass',
    'BANKSY': 'leiden_banksy_08',
    'scGPT-spatial': 'leiden_scgptspatial',
    'OmiCLIP text': 'leiden_omiclip_text',
    'OmiCLIP image': 'leiden_omiclip_image',
    'Scanpy': 'leiden_scanpy',
}

results_df = compute_all_metrics(adata, clustering_keys, ground_truth_key='region_annotation', spatial_key='spatial')


In [ ]:
plot_benchmark_heatmap(results_df, title="LUAD Benchmark", savefig='../../../SpatialFusion/results/figures_Fig4/LUAD_HD_benchmark.svg')

## Plot the clusters in space

In [ ]:
def plot_spatial_clusters_panel(
    adata,
    method_mapping,
    color_dict,
    coord_keys=("X_coord", "Y_coord"),
    ncols=5,
    savefig=None,
    rasterize_points=True,
):
    """
    Plot spatial clustering panels for multiple methods in a Nature Genetics style.
    """

    x_key, y_key = coord_keys
    method_keys = list(method_mapping.values())
    method_titles = list(method_mapping.keys())
    n_methods = len(method_keys)
    nrows = math.ceil(n_methods / ncols)

    # --- Shared color palette across all cluster IDs (as strings) ---
    all_labels = np.unique(
        np.concatenate([
            adata.obs[k].astype(str).values for k in method_keys
        ])
    )

    # --- Figure style ---
    sns.set_style("white")
    sns.set_context("talk", font_scale=1.3)

    fig, axes = plt.subplots(
        nrows=nrows, ncols=ncols, figsize=(5.5 * ncols, 5 * nrows), dpi=300
    )
    axes = np.array(axes).reshape(-1)

    for i, (display_name, method) in enumerate(method_mapping.items()):
        ax = axes[i]

        # --- Convert hue column to string to match color_dict keys ---
        hue_values = adata.obs[method].astype(str)

        sns.scatterplot(
            x=adata.obs[x_key],
            y=adata.obs[y_key],
            hue=hue_values,
            palette=color_dict[method],
            s=1,
            linewidth=0,
            alpha=0.9,
            ax=ax,
            legend=False,
            rasterized=rasterize_points,
        )

        ax.invert_yaxis()
        ax.set_aspect("equal")

        # --- Titles & styling ---
        ax.set_title(display_name, fontsize=14, fontweight="normal", pad=10)
        ax.set_xlabel("", fontsize=18, labelpad=8, fontweight="normal")
        ax.set_ylabel("", fontsize=18, labelpad=8, fontweight="normal")
        ax.set_xticks([]); ax.set_yticks([])
        for spine in ["top", "right", "left", "bottom"]:
            ax.spines[spine].set_visible(False)
        ax.grid(False)

    # Hide any unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)

    plt.tight_layout()

    # --- Shared legend ---
    rep_method = list(color_dict.keys())[0]
    rep_palette = color_dict[rep_method]
    
    handles = []
    for label in sorted(all_labels):
        if label in rep_palette:
            handles.append(
                plt.Line2D(
                    [0], [0],
                    marker="o",
                    color="none",
                    markerfacecolor=rep_palette[label],
                    markersize=8,
                    label=label,
                )
            )
    
    legend_fig = plt.figure(figsize=(2.5, 0.4 * len(handles)), dpi=300)
    legend_fig.legend(
        handles=handles,
        loc="center",
        title="Cluster",
        frameon=False,
        ncol=1,
        fontsize=12,
        title_fontsize=14,
    )
    legend_fig.tight_layout()
    
    if savefig:
        fig.savefig(
            f"{savefig}_panel.png",
            dpi=200,
            bbox_inches="tight",
            transparent=True,
        )
        legend_fig.savefig(
            f"{savefig}_legend.svg",
            dpi=300,
            bbox_inches="tight",
            transparent=True,
        )
        print(f"Saved: {savefig}_panel.png and {savefig}_legend.svg")
    
    plt.show()
    plt.close(legend_fig)

In [ ]:
leiden_cols = [
       'leiden_gcn', 'leiden_gcn_he',
       'leiden_gcn_finetuned', 'leiden_nichecompass', 
       'leiden_scgptspatial',
       'leiden_scanpy', 'leiden_omiclip_text',
       'leiden_omiclip_image', 'leiden_banksy_08']

In [ ]:
palette_specs = {
            l: tab20_filtered for l in leiden_cols
        }

palette_dict_2 = build_palettes_from_adata(adata, palette_specs)

In [ ]:
method_keys = list(clustering_keys.values())

plot_spatial_clusters_panel(
    adata,
    color_dict=palette_dict_2,
    method_mapping=clustering_keys,
    ncols=5,
    savefig='../../../SpatialFusion/results/figures_Fig4/LUAD_HD_panel_viz_clusters.svg',
)


# Co-occurrence

In [ ]:
# Step 1: Compute the cross-tabulation (co-occurrence counts)
heatmap_data = pd.crosstab(adata.obs['region_annotation'], adata.obs['leiden_gcn'])
heatmap_data = heatmap_data.div(heatmap_data.sum(axis=1), axis=0)*100
heatmap_data = heatmap_data.astype(int)

# Step 2: Plot the heatmap

fig, ax = plt.subplots(1,1,figsize=(7,5))
sns.heatmap(heatmap_data, annot=True, fmt="d", cmap="Blues",ax=ax)
ax.set_xlabel('SpatialFusion niches')
ax.set_ylabel('Region annotations')
ax.set_title('SpatialFusion co-occurrence')
fig.savefig('../../../SpatialFusion/results/figures_Fig4/LUAD_SpatialFusion_confusion.svg', dpi=200, bbox_inches='tight')
plt.show()


In [ ]:
# Step 1: Compute the cross-tabulation (co-occurrence counts)
heatmap_data = pd.crosstab(adata.obs['region_annotation'], adata.obs['leiden_nichecompass'])
heatmap_data = heatmap_data.div(heatmap_data.sum(axis=1), axis=0)*100
heatmap_data = heatmap_data.astype(int)

# Step 2: Plot the heatmap

fig, ax = plt.subplots(1,1,figsize=(7,5))
sns.heatmap(heatmap_data, annot=True, fmt="d", cmap="Blues",ax=ax)
ax.set_xlabel('NicheCompass niches')
ax.set_ylabel('Region annotations')
ax.set_title('NicheCompass co-occurrence')
fig.savefig('../../../SpatialFusion/results/figures_Fig4/LUAD_NicheCompass_confusion.svg', dpi=200, bbox_inches='tight')
plt.show()


In [ ]:
# Step 1: Compute the cross-tabulation (co-occurrence counts)
heatmap_data = pd.crosstab(adata.obs['region_annotation'], adata.obs['leiden_gcn_finetuned'])
heatmap_data = heatmap_data.div(heatmap_data.sum(axis=1), axis=0)*100
heatmap_data = heatmap_data.astype(int)

# Step 2: Plot the heatmap

fig, ax = plt.subplots(1,1,figsize=(7,5))
sns.heatmap(heatmap_data, annot=True, fmt="d", cmap="Blues",ax=ax)
ax.set_xlabel('SpatialFusion (finetuned) niches')
ax.set_ylabel('Region annotations')
ax.set_title('SpatialFusion (finetuned) co-occurrence')
fig.savefig('../../../SpatialFusion/results/figures_Fig4/LUAD_SpatialFusion_finetuned_confusion.svg', dpi=200, bbox_inches='tight')
plt.show()


In [ ]:
# Step 1: Compute the cross-tabulation (co-occurrence counts)
heatmap_data = pd.crosstab(adata.obs['region_annotation'], adata.obs['leiden_omiclip_image'])
heatmap_data = heatmap_data.div(heatmap_data.sum(axis=1), axis=0)*100
heatmap_data = heatmap_data.astype(int)

# Step 2: Plot the heatmap

fig, ax = plt.subplots(1,1,figsize=(7,5))
sns.heatmap(heatmap_data, annot=True, fmt="d", cmap="Blues",ax=ax)
ax.set_xlabel('OmiCLIP image niches')
ax.set_ylabel('Region annotations')
ax.set_title('OmiCLIP image co-occurrence')
fig.savefig('../../../SpatialFusion/results/figures_Fig4/LUAD_OmiCLIP_image_confusion.svg', dpi=200, bbox_inches='tight')
plt.show()
